In [14]:
from itertools import combinations
import networkx as nx
import random

In [29]:
def read_signed_graph_from_file(file_path):
    graph = {}
    with open(file_path, 'r') as file:
        for line in file:
            source, target, weight = map(int, line.strip().split())
            if source not in graph:
                graph[source] = []
            if target not in graph:
                graph[target] = []
            graph[source].append((target, weight))
            graph[target].append((source, weight))
    return graph

def add_graph_edges(file_path):
    Grp = nx.Graph()
    graph_list = list()
    with open(file_path, 'r') as file:
        for line in file:
            source, target, weight = map(int, line.strip().split())
            val = (source, target, {'sign':weight})
            graph_list.append(val)
            
    Grp.add_edges_from(graph_list)
    return Grp
    

def find_pos_and_neg(graph):
    for node in graph:
        pos[node] = []
        neg[node] = []
        for nei, sign in graph[node]:
            if sign == 1:
                pos[node].append(nei)
            else:
                neg[node].append(nei)
        
        for node in pos[node]:
            if node not in all_pos_node:
                all_pos_node.append(node)
        

def generate_cluster(clu, all_pos_node):
    index = 0
    is_checks = True
    while is_checks:
        start_node = random.choice(all_pos_node)
        clu.append(pos[start_node])
        for vlu in clu[index]:
            if vlu in all_pos_node: 
                all_pos_node.remove(vlu) 
        
        for nodes in clu[index]:
            for value in pos[nodes]:
                if value not in clu[index]:
                    clu[index].append(value)
                    if value in all_pos_node: 
                        all_pos_node.remove(value)
                  
        if not all_pos_node:
            is_checks = False  
        index += 1              
    
def check_balanced(clu, neg):
    for cluster_set in clu:
        for nodes in cluster_set:
            for neg_nei in neg[nodes]:
                for checking_node in cluster_set:
                    if neg_nei == checking_node:
                        return 0
    
    if len(clu) <= 2:
        return 1
    else:
        return 2
    
    
def check_condition2(s, sbar, pos_count = 0):
    is_check = False
    for s_node in s:
#         print("checking conditon 2 for s node", s_node)
        if s_node not in asd:
            neg_count = 0
            for sbar_node in sbar:
                for nei, sign in graph[sbar_node]:
                    if nei == s_node:
                        if sign == -1:
                            neg_count += 1
            if pos_count + 1 >= neg_count:
#                 print("condition 2 statisfied")
                is_check = True
    
    if is_check:
        asd.append(s) 
            
def check_condition1(s):
    is_statisfied = False
    for vertex1 in s:
#         print("checking conditon 1 for vertex" , vertex1)
        pos_count = 0
        neg_count = 0
        for nei, sign in graph[vertex1]:
            for vertex2 in s:
                if nei == vertex2:
                    if sign == 1:
                        pos_count += 1
                    else:
                        neg_count += 1
                    break
        if pos_count + 1 >= neg_count:
#             print("condition 1 statisfied")
            is_statisfied =  True
        else:
            is_statisfied =  False
            break
    return is_statisfied, pos_count

def is_complete(graph):
    num_nodes = len(graph.nodes)
    return all(len(graph.edges(node)) == num_nodes - 1 for node in graph.nodes)

In [33]:
file_path = r"/Users/mac/Desktop/Desktop/RCS_Signed_Graph/Meeting_2/graph_check/ads_graph_check4.txt"
graph = read_signed_graph_from_file(file_path)
# print(graph)

G = add_graph_edges(file_path)

if not is_complete(G):
    print("Given graph is Not Complete.")
else:
    pos = dict()
    neg = dict()
    all_pos_node = []
    clu = list(list())
    find_pos_and_neg(graph)
    generate_cluster(clu, all_pos_node)
    is_balanced = check_balanced(clu, neg)

    if(is_balanced == 0):
        print("Given graph is not balanced.")
    else:
        nodes = list(graph.keys())
        asd = list()
        # Generate all 6 combinations
        for i in range(1, len(nodes)):
            if not asd:
                s = list(combinations(nodes, i))
                for combo in s:
                    s = list(combo) 
                    sbar = [x for x in nodes if x not in s]
        #             print(s,sbar)
                    if len(s) > 1:
        #                 print("combination length greater 1")
                        is_checked, pos_count = check_condition1(s)
                        if is_checked:
                            check_condition2(s, sbar, pos_count)
                    else:
                        check_condition2(s,sbar) 
            else:
                break
             
        if not asd:
            print("There is no Smallest Defensive Alliance for this graph")
        else:
            print("Smallest Defensive Alliance")
            print(asd) 

Given graph is Not Complete.
